In [1]:
import pandas as pd

def calculate_f_score(data):
    """
    피오트로스키의 F-Score를 계산하는 함수.
    
    :param data: 재무 데이터가 포함된 데이터프레임 (index는 연도, columns는 해당 지표들)
    :return: 각 연도별 F-Score를 포함한 데이터프레임
    """
    f_score = pd.DataFrame(index=data.index)
    
    # 수익성 지표
    f_score['ROA'] = (data['net_income'] / data['total_assets']).apply(lambda x: 1 if x > 0 else 0)
    f_score['CFO'] = data['cash_flow_operations'].apply(lambda x: 1 if x > 0 else 0)
    f_score['change_ROA'] = (data['net_income'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    
    # 레버리지, 유동성, 자금조달 지표
    f_score['change_leverage'] = (data['long_term_debt'] / data['total_assets']).diff().apply(lambda x: 1 if x < 0 else 0)
    f_score['change_liquidity'] = (data['current_assets'] / data['current_liabilities']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['equity_offering'] = data['shares_outstanding'].diff().apply(lambda x: 1 if x <= 0 else 0)
    
    # 운영 효율성 지표
    f_score['change_margin'] = (data['gross_margin'] / data['total_revenue']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_turnover'] = (data['total_revenue'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    
    f_score['F_Score'] = f_score.sum(axis=1)
    
    return f_score

# 백테스팅을 위한 예제 사용법
data = pd.DataFrame({
    'net_income': [100, 150, 200],
    'total_assets': [1000, 1100, 1200],
    'cash_flow_operations': [120, 130, 140],
    'long_term_debt': [400, 350, 300],
    'current_assets': [500, 550, 600],
    'current_liabilities': [200, 220, 250],
    'shares_outstanding': [1000, 1000, 1000],
    'gross_margin': [400, 450, 500],
    'total_revenue': [1000, 1100, 1200]
}, index=[2018, 2019, 2020])

f_score_df = calculate_f_score(data)
print(f_score_df)


      ROA  CFO  change_ROA  change_leverage  change_liquidity  \
2018    1    1           0                0                 0   
2019    1    1           1                1                 0   
2020    1    1           1                1                 0   

      equity_offering  change_margin  change_turnover  F_Score  
2018                0              0                0        2  
2019                1              1                0        6  
2020                1              1                0        6  


In [3]:
import pandas as pd
import numpy as np

def calculate_f_score(data):
    f_score = pd.DataFrame(index=data.index)
    f_score['ROA'] = (data['net_income'] / data['total_assets']).apply(lambda x: 1 if x > 0 else 0)
    f_score['CFO'] = data['cash_flow_operations'].apply(lambda x: 1 if x > 0 else 0)
    f_score['change_ROA'] = (data['net_income'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_leverage'] = (data['long_term_debt'] / data['total_assets']).diff().apply(lambda x: 1 if x < 0 else 0)
    f_score['change_liquidity'] = (data['current_assets'] / data['current_liabilities']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['equity_offering'] = data['shares_outstanding'].diff().apply(lambda x: 1 if x <= 0 else 0)
    f_score['change_margin'] = (data['gross_margin'] / data['total_revenue']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_turnover'] = (data['total_revenue'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['F_Score'] = f_score.sum(axis=1)
    return f_score

def backtest_strategy(price_data, f_score_df, threshold=5):
    returns = price_data.pct_change().shift(-1).fillna(0)
    portfolio_returns = pd.Series(index=returns.index)

    for year in f_score_df.index[:-1]:
        selected_stocks = f_score_df.loc[year][f_score_df.loc[year] >= threshold].index
        if len(selected_stocks) > 0:
            selected_stocks = [stock for stock in selected_stocks if stock in returns.columns]
            if selected_stocks:
                portfolio_returns.loc[str(year+1)] = returns[selected_stocks].mean(axis=1)
    
    portfolio_cumulative_returns = (1 + portfolio_returns.dropna()).cumprod() - 1
    return portfolio_returns.dropna(), portfolio_cumulative_returns

def calculate_sharpe_ratio(returns, risk_free_rate=0.02):
    excess_returns = returns - risk_free_rate / 252  # 일일 수익률로 가정
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)
    return sharpe_ratio * np.sqrt(252)  # 연간 샤프비율로 변환

# 예제 사용법
data = pd.DataFrame({
    'net_income': [100, 150, 200],
    'total_assets': [1000, 1100, 1200],
    'cash_flow_operations': [120, 130, 140],
    'long_term_debt': [400, 350, 300],
    'current_assets': [500, 550, 600],
    'current_liabilities': [200, 220, 250],
    'shares_outstanding': [1000, 1000, 1000],
    'gross_margin': [400, 450, 500],
    'total_revenue': [1000, 1100, 1200]
}, index=[2018, 2019, 2020])

price_data = pd.DataFrame({
    'AAPL': [150, 155, 160, 165, 170, 175, 180],
    'MSFT': [100, 102, 104, 106, 108, 110, 112],
    'GOOGL': [1200, 1220, 1240, 1260, 1280, 1300, 1320]
}, index=pd.date_range(start='2020-01-01', periods=7, freq='Y'))

f_score_df = calculate_f_score(data)
f_score_df['AAPL'] = [6, 5, 7]
f_score_df['MSFT'] = [4, 6, 5]
f_score_df['GOOGL'] = [7, 8, 6]

portfolio_returns, portfolio_cumulative_returns = backtest_strategy(price_data, f_score_df)

# 샤프비율 계산
sharpe_ratio = calculate_sharpe_ratio(portfolio_returns)
print(f"Sharpe Ratio: {sharpe_ratio}")

print("Portfolio Cumulative Returns:")
print(portfolio_cumulative_returns)


C:\Users\JYB\AppData\Local\Temp/ipykernel_24772/2217281878.py:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  portfolio_returns = pd.Series(index=returns.index)


TypeError: cannot convert the series to <class 'float'>

In [4]:
import pandas as pd
import numpy as np

def calculate_f_score(data):
    f_score = pd.DataFrame(index=data.index)
    f_score['ROA'] = (data['net_income'] / data['total_assets']).apply(lambda x: 1 if x > 0 else 0)
    f_score['CFO'] = data['cash_flow_operations'].apply(lambda x: 1 if x > 0 else 0)
    f_score['change_ROA'] = (data['net_income'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_leverage'] = (data['long_term_debt'] / data['total_assets']).diff().apply(lambda x: 1 if x < 0 else 0)
    f_score['change_liquidity'] = (data['current_assets'] / data['current_liabilities']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['equity_offering'] = data['shares_outstanding'].diff().apply(lambda x: 1 if x <= 0 else 0)
    f_score['change_margin'] = (data['gross_margin'] / data['total_revenue']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_turnover'] = (data['total_revenue'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['F_Score'] = f_score.sum(axis=1)
    return f_score

def backtest_strategy(price_data, f_score_df, threshold=5):
    returns = price_data.pct_change().shift(-1).fillna(0)
    portfolio_returns = pd.Series(index=returns.index)

    for year in f_score_df.index[:-1]:
        selected_stocks = f_score_df.columns[f_score_df.loc[year] >= threshold]
        selected_stocks = [stock for stock in selected_stocks if stock in returns.columns]
        if selected_stocks:
            portfolio_returns.loc[str(year+1)] = returns[selected_stocks].mean(axis=1)
    
    portfolio_cumulative_returns = (1 + portfolio_returns.dropna()).cumprod() - 1
    return portfolio_returns.dropna(), portfolio_cumulative_returns

def calculate_sharpe_ratio(returns, risk_free_rate=0.02):
    excess_returns = returns - risk_free_rate / 252  # 일일 수익률로 가정
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)
    return sharpe_ratio * np.sqrt(252)  # 연간 샤프비율로 변환

# 예제 사용법
data = pd.DataFrame({
    'net_income': [100, 150, 200],
    'total_assets': [1000, 1100, 1200],
    'cash_flow_operations': [120, 130, 140],
    'long_term_debt': [400, 350, 300],
    'current_assets': [500, 550, 600],
    'current_liabilities': [200, 220, 250],
    'shares_outstanding': [1000, 1000, 1000],
    'gross_margin': [400, 450, 500],
    'total_revenue': [1000, 1100, 1200]
}, index=[2018, 2019, 2020])

price_data = pd.DataFrame({
    'AAPL': [150, 155, 160, 165, 170, 175, 180],
    'MSFT': [100, 102, 104, 106, 108, 110, 112],
    'GOOGL': [1200, 1220, 1240, 1260, 1280, 1300, 1320]
}, index=pd.date_range(start='2020-01-01', periods=7, freq='Y'))

f_score_df = calculate_f_score(data)
f_score_df['AAPL'] = [6, 5, 7]
f_score_df['MSFT'] = [4, 6, 5]
f_score_df['GOOGL'] = [7, 8, 6]

portfolio_returns, portfolio_cumulative_returns = backtest_strategy(price_data, f_score_df)

# 샤프비율 계산
if not portfolio_returns.empty:
    sharpe_ratio = calculate_sharpe_ratio(portfolio_returns)
    print(f"Sharpe Ratio: {sharpe_ratio}")

print("Portfolio Cumulative Returns:")
print(portfolio_cumulative_returns)


C:\Users\JYB\AppData\Local\Temp/ipykernel_24772/3746290014.py:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  portfolio_returns = pd.Series(index=returns.index)


TypeError: cannot convert the series to <class 'float'>

In [5]:
import pandas as pd
import numpy as np

def calculate_f_score(data):
    f_score = pd.DataFrame(index=data.index)
    f_score['ROA'] = (data['net_income'] / data['total_assets']).apply(lambda x: 1 if x > 0 else 0)
    f_score['CFO'] = data['cash_flow_operations'].apply(lambda x: 1 if x > 0 else 0)
    f_score['change_ROA'] = (data['net_income'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_leverage'] = (data['long_term_debt'] / data['total_assets']).diff().apply(lambda x: 1 if x < 0 else 0)
    f_score['change_liquidity'] = (data['current_assets'] / data['current_liabilities']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['equity_offering'] = data['shares_outstanding'].diff().apply(lambda x: 1 if x <= 0 else 0)
    f_score['change_margin'] = (data['gross_margin'] / data['total_revenue']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_turnover'] = (data['total_revenue'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['F_Score'] = f_score.sum(axis=1)
    return f_score

def backtest_strategy(price_data, f_score_df, threshold=5):
    returns = price_data.pct_change().shift(-1).fillna(0)
    portfolio_returns = pd.Series(dtype=float)

    for year in f_score_df.index[:-1]:
        selected_stocks = f_score_df.columns[f_score_df.loc[year] >= threshold]
        selected_stocks = [stock for stock in selected_stocks if stock in returns.columns]
        if selected_stocks:
            # 포트폴리오 수익률을 계산하고 이를 float으로 변환
            portfolio_returns.loc[str(year+1)] = returns[selected_stocks].mean(axis=1).astype(float)
    
    portfolio_cumulative_returns = (1 + portfolio_returns.dropna()).cumprod() - 1
    return portfolio_returns.dropna(), portfolio_cumulative_returns

def calculate_sharpe_ratio(returns, risk_free_rate=0.02):
    excess_returns = returns - risk_free_rate / 252  # 일일 수익률로 가정
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)
    return sharpe_ratio * np.sqrt(252)  # 연간 샤프비율로 변환

# 예제 사용법
data = pd.DataFrame({
    'net_income': [100, 150, 200],
    'total_assets': [1000, 1100, 1200],
    'cash_flow_operations': [120, 130, 140],
    'long_term_debt': [400, 350, 300],
    'current_assets': [500, 550, 600],
    'current_liabilities': [200, 220, 250],
    'shares_outstanding': [1000, 1000, 1000],
    'gross_margin': [400, 450, 500],
    'total_revenue': [1000, 1100, 1200]
}, index=[2018, 2019, 2020])

price_data = pd.DataFrame({
    'AAPL': [150, 155, 160, 165, 170, 175, 180],
    'MSFT': [100, 102, 104, 106, 108, 110, 112],
    'GOOGL': [1200, 1220, 1240, 1260, 1280, 1300, 1320]
}, index=pd.date_range(start='2020-01-01', periods=7, freq='Y'))

f_score_df = calculate_f_score(data)
f_score_df['AAPL'] = [6, 5, 7]
f_score_df['MSFT'] = [4, 6, 5]
f_score_df['GOOGL'] = [7, 8, 6]

portfolio_returns, portfolio_cumulative_returns = backtest_strategy(price_data, f_score_df)

# 샤프비율 계산
if not portfolio_returns.empty:
    sharpe_ratio = calculate_sharpe_ratio(portfolio_returns)
    print(f"Sharpe Ratio: {sharpe_ratio}")

print("Portfolio Cumulative Returns:")
print(portfolio_cumulative_returns)


TypeError: cannot convert the series to <class 'float'>

In [6]:
import pandas as pd
import numpy as np

def calculate_f_score(data):
    f_score = pd.DataFrame(index=data.index)
    f_score['ROA'] = (data['net_income'] / data['total_assets']).apply(lambda x: 1 if x > 0 else 0)
    f_score['CFO'] = data['cash_flow_operations'].apply(lambda x: 1 if x > 0 else 0)
    f_score['change_ROA'] = (data['net_income'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_leverage'] = (data['long_term_debt'] / data['total_assets']).diff().apply(lambda x: 1 if x < 0 else 0)
    f_score['change_liquidity'] = (data['current_assets'] / data['current_liabilities']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['equity_offering'] = data['shares_outstanding'].diff().apply(lambda x: 1 if x <= 0 else 0)
    f_score['change_margin'] = (data['gross_margin'] / data['total_revenue']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_turnover'] = (data['total_revenue'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['F_Score'] = f_score.sum(axis=1)
    return f_score

def backtest_strategy(price_data, f_score_df, threshold=5):
    returns = price_data.pct_change().shift(-1).fillna(0)
    portfolio_returns = pd.Series(dtype=float)

    for year in f_score_df.index[:-1]:
        selected_stocks = f_score_df.columns[f_score_df.loc[year] >= threshold]
        selected_stocks = [stock for stock in selected_stocks if stock in returns.columns]
        if selected_stocks:
            for date in returns.index:
                portfolio_returns.at[date] = returns.loc[date, selected_stocks].mean()
    
    portfolio_cumulative_returns = (1 + portfolio_returns.dropna()).cumprod() - 1
    return portfolio_returns.dropna(), portfolio_cumulative_returns

def calculate_sharpe_ratio(returns, risk_free_rate=0.02):
    excess_returns = returns - risk_free_rate / 252  # 일일 수익률로 가정
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)
    return sharpe_ratio * np.sqrt(252)  # 연간 샤프비율로 변환

# 예제 사용법
data = pd.DataFrame({
    'net_income': [100, 150, 200],
    'total_assets': [1000, 1100, 1200],
    'cash_flow_operations': [120, 130, 140],
    'long_term_debt': [400, 350, 300],
    'current_assets': [500, 550, 600],
    'current_liabilities': [200, 220, 250],
    'shares_outstanding': [1000, 1000, 1000],
    'gross_margin': [400, 450, 500],
    'total_revenue': [1000, 1100, 1200]
}, index=[2018, 2019, 2020])

price_data = pd.DataFrame({
    'AAPL': [150, 155, 160, 165, 170, 175, 180],
    'MSFT': [100, 102, 104, 106, 108, 110, 112],
    'GOOGL': [1200, 1220, 1240, 1260, 1280, 1300, 1320]
}, index=pd.date_range(start='2020-01-01', periods=7, freq='Y'))

f_score_df = calculate_f_score(data)
f_score_df['AAPL'] = [6, 5, 7]
f_score_df['MSFT'] = [4, 6, 5]
f_score_df['GOOGL'] = [7, 8, 6]

portfolio_returns, portfolio_cumulative_returns = backtest_strategy(price_data, f_score_df)

# 샤프비율 계산
if not portfolio_returns.empty:
    sharpe_ratio = calculate_sharpe_ratio(portfolio_returns)
    print(f"Sharpe Ratio: {sharpe_ratio}")

print("Portfolio Cumulative Returns:")
print(portfolio_cumulative_returns)


Sharpe Ratio: 38.49781619129201
Portfolio Cumulative Returns:
2020-12-31    0.023333
2021-12-31    0.046617
2022-12-31    0.069856
2023-12-31    0.093052
2024-12-31    0.116208
2025-12-31    0.139328
2026-12-31    0.139328
dtype: float64
